In [79]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_contemp_matrices, transformation, standardize
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [68]:
# Set M and standardization

M = 3
standardization = False
transform = True

In [69]:
ds = pd.read_csv("../data/fred_qd.csv")
gdp = transformation(ds["GDPC1"].iloc[2:].to_numpy(), 5, transform, scale=1)
cpi = transformation(ds["CPIAUCSL"].iloc[2:].to_numpy(), 6, transform, scale=1)
fedfund = transformation(ds["FEDFUNDS"].iloc[2:].to_numpy(), 2, transform, scale=1)
compi = transformation(ds["PPIACO"].iloc[2:].to_numpy(), 6, transform, scale=1)
borrowings = transformation(ds["TOTRESNS"].iloc[2:].to_numpy(), 6, transform, scale=1)
sp500 = transformation(ds["S&P 500"].iloc[2:].to_numpy(), 5, transform, scale=1)
m2 = transformation(ds["M2REAL"].iloc[2:].to_numpy(), 5, transform, scale=1)

# Start due to transformation

lag = 1

if M == 3:

    series = [gdp[lag:], cpi[lag:], fedfund[lag:]]
    
elif M == 7:
    
    series = [gdp[lag:], cpi[lag:], fedfund[lag:],compi[lag:],borrowings[lag:],sp500[lag:],m2[lag:]]
    

if standardization:
    
    series = standardize(series, train = 243-25)

series_total = np.array(series)

y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(244, M, 1, series_total)

In [70]:
prior_parameters= {"a0":4,"b0":1}

In [75]:
# Check if the model, given the specific data, converges at all

T = 243
p = 1
prior = "lasso_alternative"
train = T - 25
prior_parameters = {"a0_lasso":1.1e-3, "b0_lasso":1e-3}

*_, y_pred, complete_msfe = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=True, iterations=25)

Iteration: 20 
 Elapsed time: 1.5031 seconds
Seconds for one iteration: 0.1692
 Difference: 17.2654
Seconds for one iteration: 0.1755
 Difference: 17.1664
Seconds for one iteration: 0.1412
 Difference: 17.0279
Seconds for one iteration: 0.1417
 Difference: 16.9471
Seconds for one iteration: 0.1325
 Difference: 16.8342
Progress: 1/26 | Elapsed time: 3.4894421100616455 seconds | Coefficients converged: False
ETA: 2020-11-30 16:17:41.294067
Progress: 2/26 | Elapsed time: 3.499069929122925 seconds | Coefficients converged: False
ETA: 2020-11-30 16:17:41.535251
Progress: 3/26 | Elapsed time: 3.5987608432769775 seconds | Coefficients converged: False
ETA: 2020-11-30 16:17:43.928338
Progress: 4/26 | Elapsed time: 3.6199228763580322 seconds | Coefficients converged: False
ETA: 2020-11-30 16:17:44.415537
Progress: 5/26 | Elapsed time: 3.7269067764282227 seconds | Coefficients converged: False
ETA: 2020-11-30 16:17:46.769681
Progress: 6/26 | Elapsed time: 3.6868700981140137 seconds | Coefficient

In [85]:
y_pred[:,0,:,0].flatten().flatten()

array([ 0.00376625, -0.00059308, -0.01302898])

In [ ]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [20]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=True, iterations=100, prior_parameters=prior_parameters)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [21]:
simulation_run(1,3,"horseshoe")

Iteration: 10 
 Elapsed time: 4.5246 seconds
lambda: 0.07983741324089205,phi: 3.1938730536986024, v: 0.6315611180509229, delta: 7.005081318625064 mt1t: 0.21777342673623507
Seconds for one iteration: 0.413
 Difference: 0.0013
lambda: 0.07981793351232738,phi: 3.1938795825028965, v: 0.6315589950736226, delta: 7.005991084062367 mt1t: 0.21777396864062934
Seconds for one iteration: 0.4198
 Difference: 0.0006
lambda: 0.07981014774586277,phi: 3.193883327937246, v: 0.6315580186594768, delta: 7.006301636863676 mt1t: 0.21777420822280916
Seconds for one iteration: 0.422
 Difference: 0.0003
lambda: 0.07980702552818858,phi: 3.1938852361058716, v: 0.631557586814332, delta: 7.006405364938962 mt1t: 0.21777431366405645
Seconds for one iteration: 0.4475
 Difference: 0.0001
Progress: 1/25 | Elapsed time: 5.542405128479004 seconds | Coefficients converged: True
ETA: 2020-11-25 18:17:18.640126
Progress: 2/25 | Elapsed time: 5.871854066848755 seconds | Coefficients converged: True
ETA: 2020-11-25 18:17:26.54

[array([1.70472609, 1.98461325, 2.04759078, 1.87832718, 1.88262929,
        1.96404813, 2.02673511, 2.09856824]),
 array([-11.46809142, -31.94019158, -25.02712076, -30.48352617,
        -36.84136149, -19.28005581, -33.34170897, -35.88657947]),
 0.01813187601112523,
 3,
 'horseshoe']